In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

import lightgbm as lgb

from imblearn.over_sampling import SMOTE

from keras.models import Sequential
from keras.layers import Dense

import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
df = pd.read_excel('Dataset.xlsx')

## Clean Data

In [ ]:
# Get the list of columns to convert to categorical
categorical_columns = df.select_dtypes(include='int64').columns.tolist()

# Convert the selected columns to categorical
df[categorical_columns] = df[categorical_columns].astype('category')

numeric_data = df.select_dtypes(include=[np.number])

# Calculate MAD for each column
mad = numeric_data.mad()

# Choose a threshold multiplier
k = 3

# Calculate the threshold value
threshold = k * mad

# Identify outliers
outliers = (np.abs(numeric_data - numeric_data.median()) > threshold)

# Apply logarithm to the specified columns
outlier_columns = ['CommissionSacrificePercentage', 'BonusCommissionPercentage']
for column in outlier_columns:
    df[column] = np.log1p(df[column])
    
df = df.drop(columns=['PropDate'])

### SMOTE

In [14]:
# Splitting the dataset into features (X) and target variable (y)
X = df.drop("PolicyIssued", axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train_80, X_test_80, y_train_80, y_test_80 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_70, X_test_70, y_train_70, y_test_70 = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled_80 = scaler.fit_transform(X_train_80)
X_test_scaled_80 = scaler.transform(X_test_80)

X_train_scaled_70 = scaler.fit_transform(X_train_70)
X_test_scaled_70 = scaler.transform(X_test_70)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled_80, y_resampled_80 = smote.fit_resample(X_train_scaled_80, y_train_80)

X_resampled_70, y_resampled_70 = smote.fit_resample(X_train_scaled_70, y_train_70)

# Convert the resampled arrays back to a dataframe
resampled_df_80 = pd.DataFrame(X_resampled_80, columns=X_train_80.columns)
resampled_df_80["PolicyIssued"] = y_resampled_80

resampled_df_70 = pd.DataFrame(X_resampled_70, columns=X_train_70.columns)
resampled_df_70["PolicyIssued"] = y_resampled_70

# Check the class distribution in the resampled dataset
print("Class distribution in resampled dataset:")
print(resampled_df_80["PolicyIssued"].value_counts())
print(resampled_df_70["PolicyIssued"].value_counts())

Class distribution in resampled dataset:
0    97922
1    97922
Name: PolicyIssued, dtype: int64
0    85682
1    85682
Name: PolicyIssued, dtype: int64


# All Features

## Logistic Regression

#### 80/20 Split

In [8]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# Initialise and train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test_scaled_80)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_train, y_pred_train)
test_auc = roc_auc_score(y_test_80, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test_80, y_pred_test)

# Evaluate the model
confusion_mat_train = confusion_matrix(y_train, y_pred_train)
class_report_train = classification_report(y_train, y_pred_train)

confusion_mat_test = confusion_matrix(y_test_80, y_pred_test)
class_report_test = classification_report(y_test_80, y_pred_test)

print("Training Confusion Matrix:\n", confusion_mat_train)
print("\nTraining Classification Report:\n", class_report_train)
print("\nTraining AUC:", train_auc)
print("Training Accuracy:", train_accuracy)
print("---------------------------------------------")

print("Test Confusion Matrix:\n", confusion_mat_test)
print("\nTest Classification Report:\n", class_report_test)
print("\nTest AUC:", test_auc)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Training Confusion Matrix:
 [[84906 13016]
 [36908 61014]]

Training Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.87      0.77     97922
           1       0.82      0.62      0.71     97922

    accuracy                           0.75    195844
   macro avg       0.76      0.75      0.74    195844
weighted avg       0.76      0.75      0.74    195844


Training AUC: 0.7450828210208124
Training Accuracy: 0.7450828210208125
---------------------------------------------
Test Confusion Matrix:
 [[ 6088   884]
 [ 9179 15302]]

Test Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.87      0.55      6972
           1       0.95      0.63      0.75     24481

    accuracy                           0.68     31453
   macro avg       0.67      0.75      0.65     31453
weighted avg       0.82      0.68      0.71     31453


Test AUC: 0.7491316400886301
Test Accuracy: 0.680

#### Cross Validation

In [6]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Initialise logistic regression model
model = LogisticRegression()

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")
    
    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_pred = model.predict(X_test_scaled_80)

    # Calculate AUC
    auc = roc_auc_score(y_test_80, y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_80, y_pred)

    # Print evaluation metrics for the test data
    print("Test AUC:", auc)
    print("Test Accuracy:", accuracy)
    print("---------------------------------------------")


Number of Folds: 5
Average AUC: 0.7451126129884722
Average Accuracy: 0.7450981408992405
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Test AUC: 0.7491316400886301
Test Accuracy: 0.6800623152004578
---------------------------------------------
Number of Folds: 10
Average AUC: 0.7452153304382247
Average Accuracy: 0.7452053720709627
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Test AUC: 0.7491316400886301
Test Accuracy: 0.6800623152004578
---------------------------------------------
Number of Folds: 20
Average AUC: 0.7451369120351143
Average Accuracy: 0.7451288328948448
---------------------------------------------
Test AUC: 0.7491316400886301
Test Accuracy: 0.6800623152004578
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


#### 70/30 Split

In [10]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# Initialise and train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test_scaled_70)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_train, y_pred_train)
test_auc = roc_auc_score(y_test_70, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test_70, y_pred_test)

# Evaluate the model
confusion_mat_train = confusion_matrix(y_train, y_pred_train)
class_report_train = classification_report(y_train, y_pred_train)

confusion_mat_test = confusion_matrix(y_test_70, y_pred_test)
class_report_test = classification_report(y_test_70, y_pred_test)

print("Training Confusion Matrix:\n", confusion_mat_train)
print("\nTraining Classification Report:\n", class_report_train)
print("\nTraining AUC:", train_auc)
print("Training Accuracy:", train_accuracy)
print("---------------------------------------------")

print("Test Confusion Matrix:\n", confusion_mat_test)
print("\nTest Classification Report:\n", class_report_test)
print("\nTest AUC:", test_auc)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Training Confusion Matrix:
 [[74548 11134]
 [32413 53269]]

Training Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.87      0.77     85682
           1       0.83      0.62      0.71     85682

    accuracy                           0.75    171364
   macro avg       0.76      0.75      0.74    171364
weighted avg       0.76      0.75      0.74    171364


Training AUC: 0.7458801148432577
Training Accuracy: 0.7458801148432577
---------------------------------------------
Test Confusion Matrix:
 [[ 9114  1344]
 [13859 22862]]

Test Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.87      0.55     10458
           1       0.94      0.62      0.75     36721

    accuracy                           0.68     47179
   macro avg       0.67      0.75      0.65     47179
weighted avg       0.82      0.68      0.70     47179


Test AUC: 0.747036237321498
Test Accuracy: 0.6777

#### Cross Validation

In [8]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Initialise logistic regression model
model = LogisticRegression()

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")
    
    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_pred = model.predict(X_test_scaled_70)

    # Calculate AUC
    auc = roc_auc_score(y_test_70, y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_70, y_pred)

    # Print evaluation metrics for the test data
    print("Test AUC:", auc)
    print("Test Accuracy:", accuracy)
    print("---------------------------------------------")


Number of Folds: 5
Average AUC: 0.7451126129884722
Average Accuracy: 0.7450981408992405
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Test AUC: 0.7479896776751963
Test Accuracy: 0.6791369041310753
---------------------------------------------
Number of Folds: 10
Average AUC: 0.7452153304382247
Average Accuracy: 0.7452053720709627
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Test AUC: 0.7479896776751963
Test Accuracy: 0.6791369041310753
---------------------------------------------
Number of Folds: 20
Average AUC: 0.7451369120351143
Average Accuracy: 0.7451288328948448
---------------------------------------------
Test AUC: 0.7479896776751963
Test Accuracy: 0.6791369041310753
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


### Stochastic Gradient Descent

#### 80/20 Split

In [9]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# Initialise SGD classifier model
model = SGDClassifier(loss='log', random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test_scaled_80)
y_train_pred = model.predict(X_train)

# Calculate AUC and accuracy for test and training data
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test_80, y_test_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test_80, y_test_pred)

# Print metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("=============================================")


Training AUC: 0.7441381916219032
Test AUC: 0.7481123499641426
Training Accuracy: 0.7441381916219032
Test Accuracy: 0.6848631291132802


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


#### Cross Validation

In [10]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Initialise SGD classifier model
model = SGDClassifier(loss='log', random_state=42)

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")

    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_test_pred = model.predict(X_test_scaled_80)
    y_train_pred = model.predict(X_train)

    # Calculate AUC and accuracy for test and training data
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_auc = roc_auc_score(y_test_80, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test_80, y_test_pred)

    # Print metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


Number of Folds: 5
Average AUC: 0.7438552938226421
Average Accuracy: 0.7438165139504831
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.7441381916219032
Test AUC: 0.7481123499641426
Training Accuracy: 0.7441381916219032
Test Accuracy: 0.6848631291132802
Number of Folds: 10
Average AUC: 0.7430257612809805
Average Accuracy: 0.7430097096972139
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.7441381916219032
Test AUC: 0.7481123499641426
Training Accuracy: 0.7441381916219032
Test Accuracy: 0.6848631291132802
Number of Folds: 20
Average AUC: 0.7436270408590044
Average Accuracy: 0.7436378667502263
---------------------------------------------
Training AUC: 0.7441381916219032
Test AUC: 0.7481123499641426
Training Accuracy: 0.7441381916219032
Test Accuracy: 0.6848631291132802


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


#### 70/30 Split

In [11]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# Initialise SGD classifier model
model = SGDClassifier(loss='log', random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test_scaled_70)
y_train_pred = model.predict(X_train)

# Calculate AUC and accuracy for test and training data
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test_70, y_test_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test_70, y_test_pred)

# Print metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("=============================================")


Training AUC: 0.7430440465908825
Test AUC: 0.744983151212081
Training Accuracy: 0.7430440465908825
Test Accuracy: 0.6829733567900973


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


#### Cross Validation

In [12]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Initialise SGD classifier model
model = SGDClassifier(loss='log', random_state=42)

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")

    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_test_pred = model.predict(X_test_scaled_70)
    y_train_pred = model.predict(X_train)

    # Calculate AUC and accuracy for test and training data
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_auc = roc_auc_score(y_test_70, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test_70, y_test_pred)

    # Print metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


Number of Folds: 5
Average AUC: 0.7446670782625764
Average Accuracy: 0.7446780083861709
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.7430440465908825
Test AUC: 0.744983151212081
Training Accuracy: 0.7430440465908825
Test Accuracy: 0.6829733567900973
Number of Folds: 10
Average AUC: 0.7442356886883358
Average Accuracy: 0.7442228423142612
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.7430440465908825
Test AUC: 0.744983151212081
Training Accuracy: 0.7430440465908825
Test Accuracy: 0.6829733567900973
Number of Folds: 20
Average AUC: 0.7447565050471809
Average Accuracy: 0.7447713889305675
---------------------------------------------
Training AUC: 0.7430440465908825
Test AUC: 0.744983151212081
Training Accuracy: 0.7430440465908825
Test Accuracy: 0.6829733567900973


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


### Decision Tree

#### 80/20 Split

In [13]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]


# Initialise decision tree classifier model
model = DecisionTreeClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test_scaled_80)
y_train_pred = model.predict(X_train)

# Calculate AUC and accuracy for test and training data
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test_80, y_test_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test_80, y_test_pred)

# Print metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("=============================================")


Training AUC: 0.8993076121811238
Test AUC: 0.6989925512269247
Training Accuracy: 0.8993076121811238
Test Accuracy: 0.7387848535910724


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


#### Cross-Validation

In [14]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# Initialise decision tree classifier model
model = DecisionTreeClassifier()

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")

    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_test_pred = model.predict(X_test_scaled_80)
    y_train_pred = model.predict(X_train)

    # Calculate AUC and accuracy for test and training data
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_auc = roc_auc_score(y_test_80, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test_80, y_test_pred)

    # Print metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


Number of Folds: 5
Average AUC: 0.7752625774716503
Average Accuracy: 0.7752445875356279
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.8993076121811238
Test AUC: 0.6990552117847173
Training Accuracy: 0.8993076121811238
Test Accuracy: 0.7384033319556164
Number of Folds: 10
Average AUC: 0.7783052570194336
Average Accuracy: 0.7782929188024882
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.8993076121811238
Test AUC: 0.6994945592590532
Training Accuracy: 0.8993076121811238
Test Accuracy: 0.7390074078784218
Number of Folds: 20
Average AUC: 0.7793871042141495
Average Accuracy: 0.7793804934113268
---------------------------------------------
Training AUC: 0.8993076121811238
Test AUC: 0.6991359820932471
Training Accuracy: 0.8993076121811238
Test Accuracy: 0.7388484405303151


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


#### 70/30 Split

In [15]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]


# Initialise decision tree classifier model
model = DecisionTreeClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test_scaled_70)
y_train_pred = model.predict(X_train)

# Calculate AUC and accuracy for test and training data
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test_70, y_test_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test_70, y_test_pred)

# Print metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("=============================================")


Training AUC: 0.9038771270511893
Test AUC: 0.6950532460612049
Training Accuracy: 0.9038771270511893
Test Accuracy: 0.7347548697513724


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


#### Cross Validation

In [16]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# Initialise decision tree classifier model
model = DecisionTreeClassifier()

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")

    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_test_pred = model.predict(X_test_scaled_70)
    y_train_pred = model.predict(X_train)

    # Calculate AUC and accuracy for test and training data
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_auc = roc_auc_score(y_test_70, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test_70, y_test_pred)

    # Print metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


Number of Folds: 5
Average AUC: 0.7770326311011321
Average Accuracy: 0.7770360057336623
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.9038771270511893
Test AUC: 0.694377554828536
Training Accuracy: 0.9038771270511893
Test Accuracy: 0.7342885605883974
Number of Folds: 10
Average AUC: 0.7797616755554133
Average Accuracy: 0.7797612090585199
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.9038771270511893
Test AUC: 0.6946979961248576
Training Accuracy: 0.9038771270511893
Test Accuracy: 0.7346276945251065
Number of Folds: 20
Average AUC: 0.7808064543141777
Average Accuracy: 0.7808116132631916
---------------------------------------------
Training AUC: 0.9038771270511893
Test AUC: 0.6942946286306492
Training Accuracy: 0.9038771270511893
Test Accuracy: 0.7345853027830178


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


### Random Forest

#### 80/20 Split

In [17]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# Initialise Random Forest classifier model
model = RandomForestClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test_scaled_80)
y_train_pred = model.predict(X_train)

# Calculate AUC and accuracy for test and training data
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test_80, y_test_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test_80, y_test_pred)

# Print metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("=============================================")


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.8992412328179572
Test AUC: 0.7020476181336361
Training Accuracy: 0.8992412328179572
Test Accuracy: 0.7500079483674054


#### Cross Validation

In [18]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# Initialise decision tree classifier model
model = DecisionTreeClassifier()

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")

    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_test_pred = model.predict(X_test_scaled_80)
    y_train_pred = model.predict(X_train)

    # Calculate AUC and accuracy for test and training data
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_auc = roc_auc_score(y_test_80, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test_80, y_test_pred)

    # Print metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


Number of Folds: 5
Average AUC: 0.7750824328926993
Average Accuracy: 0.7750658752995441
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.8993076121811238
Test AUC: 0.6996883558556295
Training Accuracy: 0.8993076121811238
Test Accuracy: 0.7393889295138778
Number of Folds: 10
Average AUC: 0.7780557964971635
Average Accuracy: 0.7780427226364548
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.8993076121811238
Test AUC: 0.6991065032155911
Training Accuracy: 0.8993076121811238
Test Accuracy: 0.7384033319556164
Number of Folds: 20
Average AUC: 0.7791310543902574
Average Accuracy: 0.7791251970319937
---------------------------------------------
Training AUC: 0.8993076121811238
Test AUC: 0.6992186038030173
Training Accuracy: 0.8993076121811238
Test Accuracy: 0.738657679712587


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


#### 70/30 Split

In [19]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# Initialise Random Forest classifier model
model = RandomForestClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test_scaled_70)
y_train_pred = model.predict(X_train)

# Calculate AUC and accuracy for test and training data
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test_70, y_test_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test_70, y_test_pred)

# Print metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("=============================================")


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.9038187717373544
Test AUC: 0.6955478177908271
Training Accuracy: 0.9038187717373544
Test Accuracy: 0.7457979185654634


#### Cross Validation

In [20]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# Initialise decision tree classifier model
model = DecisionTreeClassifier()

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")

    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_test_pred = model.predict(X_test_scaled_70)
    y_train_pred = model.predict(X_train)

    # Calculate AUC and accuracy for test and training data
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_auc = roc_auc_score(y_test_70, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test_70, y_test_pred)

    # Print metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


Number of Folds: 5
Average AUC: 0.7775006933129462
Average Accuracy: 0.7775028480741346
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.9038771270511893
Test AUC: 0.6941935826705317
Training Accuracy: 0.9038771270511893
Test Accuracy: 0.7340554060069099
Number of Folds: 10
Average AUC: 0.7798253614534119
Average Accuracy: 0.7798254020831845
---------------------------------------------


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Training AUC: 0.9038771270511893
Test AUC: 0.6943227828117567
Training Accuracy: 0.9038771270511893
Test Accuracy: 0.7343097564594417
Number of Folds: 20
Average AUC: 0.7810927090968591
Average Accuracy: 0.7810975514413181
---------------------------------------------
Training AUC: 0.9038771270511893
Test AUC: 0.6942059697811059
Training Accuracy: 0.9038771270511893
Test Accuracy: 0.7345005192988406


C:\Users\gregl\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


### LightGBM

#### 80/20 Split

In [21]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# Initialise LightGBM classifier model
model = lgb.LGBMClassifier(random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test_scaled_80)
y_train_pred = model.predict(X_train)

# Calculate AUC and accuracy for test and training data
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test_80, y_test_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test_80, y_test_pred)

# Print metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("=============================================")


Training AUC: 0.7676109556585854
Test AUC: 0.7505888627716325
Training Accuracy: 0.7676109556585854
Test Accuracy: 0.7010142116809207


#### Cross Validation

In [22]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# Initialise LightGBM classifier model
model = lgb.LGBMClassifier(random_state=42)

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")

    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Initialise LightGBM classifier model
        model = lgb.LGBMClassifier(random_state=42)

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_test_pred = model.predict(X_test_scaled_80)
    y_train_pred = model.predict(X_train)

    # Calculate AUC and accuracy for test and training data
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_auc = roc_auc_score(y_test_80, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test_80, y_test_pred)

    # Print metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


Number of Folds: 5
Average AUC: 0.7648755892156187
Average Accuracy: 0.7648587599266092
---------------------------------------------
Training AUC: 0.7676109556585854
Test AUC: 0.7505888627716325
Training Accuracy: 0.7676109556585854
Test Accuracy: 0.7010142116809207
Number of Folds: 10
Average AUC: 0.7655487824404748
Average Accuracy: 0.7655429778817322
---------------------------------------------
Training AUC: 0.7676109556585854
Test AUC: 0.7505888627716325
Training Accuracy: 0.7676109556585854
Test Accuracy: 0.7010142116809207
Number of Folds: 20
Average AUC: 0.7649567222737803
Average Accuracy: 0.7649404973598922
---------------------------------------------
Training AUC: 0.7676109556585854
Test AUC: 0.7505888627716325
Training Accuracy: 0.7676109556585854
Test Accuracy: 0.7010142116809207


#### 70/30 Split

In [23]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# Initialise LightGBM classifier model
model = lgb.LGBMClassifier(random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test_scaled_70)
y_train_pred = model.predict(X_train)

# Calculate AUC and accuracy for test and training data
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test_70, y_test_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test_70, y_test_pred)

# Print metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("=============================================")


Training AUC: 0.769549030134684
Test AUC: 0.7512973656013996
Training Accuracy: 0.769549030134684
Test Accuracy: 0.7011594141461244


#### Cross Validation

In [24]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# Initialise LightGBM classifier model
model = lgb.LGBMClassifier(random_state=42)

# Lists to store evaluation metrics for each fold
auc_scores = []
accuracy_scores = []

for num_folds in num_folds_list:
    print(f"Number of Folds: {num_folds}")

    # Initialise k-fold cross-validation
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Lists to store evaluation metrics for each fold
    fold_auc_scores = []
    fold_accuracy_scores = []

    # Perform k-fold cross-validation
    for train_index, val_index in kf.split(X_train):
        X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
        X_fold_val, y_fold_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # Initialise LightGBM classifier model
        model = lgb.LGBMClassifier(random_state=42)

        # Train the model on the fold training data
        model.fit(X_fold_train, y_fold_train)

        # Predict on the fold validation data
        y_fold_val_pred = model.predict(X_fold_val)

        # Calculate AUC and accuracy for the fold
        fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

        fold_auc_scores.append(fold_auc)
        fold_accuracy_scores.append(fold_accuracy)

    # Calculate average AUC and accuracy for the current number of folds
    avg_auc = sum(fold_auc_scores) / num_folds
    avg_accuracy = sum(fold_accuracy_scores) / num_folds

    # Print average metrics for cross-validation
    print("Average AUC:", avg_auc)
    print("Average Accuracy:", avg_accuracy)
    print("---------------------------------------------")
    
    # Train the model on the entire training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_test_pred = model.predict(X_test_scaled_70)
    y_train_pred = model.predict(X_train)

    # Calculate AUC and accuracy for test and training data
    train_auc = roc_auc_score(y_train, y_train_pred)
    test_auc = roc_auc_score(y_test_70, y_test_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test_70, y_test_pred)

    # Print metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


Number of Folds: 5
Average AUC: 0.7673676751085572
Average Accuracy: 0.7673607174466086
---------------------------------------------
Training AUC: 0.769549030134684
Test AUC: 0.7512973656013996
Training Accuracy: 0.769549030134684
Test Accuracy: 0.7011594141461244
Number of Folds: 10
Average AUC: 0.767218307971391
Average Accuracy: 0.7672148366650544
---------------------------------------------
Training AUC: 0.769549030134684
Test AUC: 0.7512973656013996
Training Accuracy: 0.769549030134684
Test Accuracy: 0.7011594141461244
Number of Folds: 20
Average AUC: 0.7674007826989743
Average Accuracy: 0.7674132602821345
---------------------------------------------
Training AUC: 0.769549030134684
Test AUC: 0.7512973656013996
Training Accuracy: 0.769549030134684
Test Accuracy: 0.7011594141461244


## Neural Network

#### 8020 Split

Loop over epochs

In [25]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# List of epochs to iterate over
epochs_list = [10, 20, 30] 

# Create empty lists to store results
results = []

for epochs in epochs_list:
    print(f" Epochs: {epochs}")

    # Initialise a neural network model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1, validation_split=0.2)

    # Evaluate the model on the separate test data
    test_loss, test_accuracy = model.evaluate(X_test_scaled_80, y_test_80, verbose=0)

    # Print metrics for test data
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


 Epochs: 10
Epoch 1/10
4897/4897 [==============================] - 7s 1ms/step - loss: 0.5184 - accuracy: 0.7280 - val_loss: 0.6957 - val_accuracy: 0.7407
Epoch 2/10
4897/4897 [==============================] - 6s 1ms/step - loss: 0.5124 - accuracy: 0.7329 - val_loss: 0.7095 - val_accuracy: 0.7148
Epoch 3/10
4897/4897 [==============================] - 6s 1ms/step - loss: 0.5110 - accuracy: 0.7352 - val_loss: 0.7522 - val_accuracy: 0.6723
Epoch 4/10
4897/4897 [==============================] - 6s 1ms/step - loss: 0.5098 - accuracy: 0.7362 - val_loss: 0.7344 - val_accuracy: 0.7036
Epoch 5/10
4897/4897 [==============================] - 6s 1ms/step - loss: 0.5091 - accuracy: 0.7361 - val_loss: 0.7233 - val_accuracy: 0.7088
Epoch 6/10
4897/4897 [==============================] - 8s 2ms/step - loss: 0.5083 - accuracy: 0.7365 - val_loss: 0.7619 - val_accuracy: 0.6642
Epoch 7/10
4897/4897 [==============================] - 6s 1ms/step - loss: 0.5077 - accuracy: 0.7372 - val_loss: 0.7002 - v

Epoch 26/30
4897/4897 [==============================] - 7s 1ms/step - loss: 0.5006 - accuracy: 0.7423 - val_loss: 0.7380 - val_accuracy: 0.6883
Epoch 27/30
4897/4897 [==============================] - 6s 1ms/step - loss: 0.5003 - accuracy: 0.7426 - val_loss: 0.7200 - val_accuracy: 0.6814
Epoch 28/30
4897/4897 [==============================] - 6s 1ms/step - loss: 0.5003 - accuracy: 0.7425 - val_loss: 0.7140 - val_accuracy: 0.6930
Epoch 29/30
4897/4897 [==============================] - 6s 1ms/step - loss: 0.4997 - accuracy: 0.7432 - val_loss: 0.6848 - val_accuracy: 0.7125
Epoch 30/30
4897/4897 [==============================] - 6s 1ms/step - loss: 0.4993 - accuracy: 0.7431 - val_loss: 0.7556 - val_accuracy: 0.6410
Test Loss: 0.44886070489883423
Test Accuracy: 0.7592916488647461


#### 70/30 Split

In [26]:
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# List of epochs to iterate over
epochs_list = [10, 20, 30] 

# Create empty lists to store results
results = []

for epochs in epochs_list:
    print(f" Epochs: {epochs}")

    # Initialise a neural network model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1, validation_split=0.2)

    # Evaluate the model on the separate test data
    test_loss, test_accuracy = model.evaluate(X_test_scaled_70, y_test_70, verbose=0)

    # Print metrics for test data
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)
    print("=============================================")


 Epochs: 10
Epoch 1/10
4285/4285 [==============================] - 7s 1ms/step - loss: 0.5171 - accuracy: 0.7302 - val_loss: 0.8075 - val_accuracy: 0.6136
Epoch 2/10
4285/4285 [==============================] - 5s 1ms/step - loss: 0.5116 - accuracy: 0.7340 - val_loss: 0.7042 - val_accuracy: 0.7247
Epoch 3/10
4285/4285 [==============================] - 6s 1ms/step - loss: 0.5099 - accuracy: 0.7360 - val_loss: 0.6825 - val_accuracy: 0.7209
Epoch 4/10
4285/4285 [==============================] - 7s 2ms/step - loss: 0.5091 - accuracy: 0.7356 - val_loss: 0.6819 - val_accuracy: 0.7378
Epoch 5/10
4285/4285 [==============================] - 6s 1ms/step - loss: 0.5081 - accuracy: 0.7367 - val_loss: 0.7000 - val_accuracy: 0.7132
Epoch 6/10
4285/4285 [==============================] - 6s 1ms/step - loss: 0.5071 - accuracy: 0.7378 - val_loss: 0.6768 - val_accuracy: 0.7512
Epoch 7/10
4285/4285 [==============================] - 6s 1ms/step - loss: 0.5063 - accuracy: 0.7368 - val_loss: 0.7152 - v

Epoch 26/30
4285/4285 [==============================] - 6s 1ms/step - loss: 0.4985 - accuracy: 0.7424 - val_loss: 0.7412 - val_accuracy: 0.6785
Epoch 27/30
4285/4285 [==============================] - 6s 1ms/step - loss: 0.4983 - accuracy: 0.7432 - val_loss: 0.6981 - val_accuracy: 0.6824
Epoch 28/30
4285/4285 [==============================] - 6s 1ms/step - loss: 0.4979 - accuracy: 0.7438 - val_loss: 0.7296 - val_accuracy: 0.6823
Epoch 29/30
4285/4285 [==============================] - 6s 1ms/step - loss: 0.4976 - accuracy: 0.7434 - val_loss: 0.6707 - val_accuracy: 0.7123
Epoch 30/30
4285/4285 [==============================] - 6s 1ms/step - loss: 0.4974 - accuracy: 0.7441 - val_loss: 0.7006 - val_accuracy: 0.6932
Test Loss: 0.45899584889411926
Test Accuracy: 0.7491680383682251


In [16]:
# List of epochs to iterate over
epochs_list = [10, 20, 30]
    
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_80.drop("PolicyIssued", axis=1)
y_train = resampled_df_80["PolicyIssued"]

# Train the model on the entire training data using a neural network
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=len(X.columns)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Loop over epochs
for epochs in epochs_list:
    print(f"Epochs: {epochs}")

    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)

    # Predict on the test data
    y_pred_train = np.round(model.predict(X_train)).astype(int)
    y_pred_test = np.round(model.predict(X_test_80)).astype(int)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_train, model.predict(X_train))
    test_auc = roc_auc_score(y_test_80, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test_80, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Epochs: 10
6121/6121 [==============================] - 6s 761us/step
Training AUC: 0.8090124812350254
Test AUC: 0.5
Training Accuracy: 0.7504595494373072
Test Accuracy: 0.7783359298000191
---------------------------------------------
Epochs: 20
6121/6121 [==============================] - 6s 853us/step
Training AUC: 0.8116293352587005
Test AUC: 0.5
Training Accuracy: 0.7513071628438962
Test Accuracy: 0.7783359298000191
---------------------------------------------
Epochs: 30
6121/6121 [==============================] - 6s 812us/step
Training AUC: 0.8139730665378064
Test AUC: 0.499959151995425
Training Accuracy: 0.7524611425420232
Test Accuracy: 0.7782723428607764
---------------------------------------------


In [17]:
# List of epochs to iterate over
epochs_list = [10, 20, 30]
    
# Splitting the dataset into features (X) and target variable (y)
X_train = resampled_df_70.drop("PolicyIssued", axis=1)
y_train = resampled_df_70["PolicyIssued"]

# Train the model on the entire training data using a neural network
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=len(X.columns)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Loop over epochs
for epochs in epochs_list:
    print(f"Epochs: {epochs}")

    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)

    # Predict on the test data
    y_pred_train = np.round(model.predict(X_train)).astype(int)
    y_pred_test = np.round(model.predict(X_test_70)).astype(int)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_train, model.predict(X_train))
    test_auc = roc_auc_score(y_test_70, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test_70, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Epochs: 10
5356/5356 [==============================] - 7s 763us/step
Training AUC: 0.8104333221646629
Test AUC: 0.499986383813077
Training Accuracy: 0.7508753297075231
Test Accuracy: 0.7783123847474512
---------------------------------------------
Epochs: 20
5356/5356 [==============================] - 5s 784us/step
Training AUC: 0.8128551136609363
Test AUC: 0.5113831687232941
Training Accuracy: 0.7523984033986135
Test Accuracy: 0.7677356450963352
---------------------------------------------
Epochs: 30
5356/5356 [==============================] - 5s 851us/step
Training AUC: 0.8164514937889974
Test AUC: 0.5005972686100895
Training Accuracy: 0.7541315562195093
Test Accuracy: 0.22323491383878422
---------------------------------------------
